
&nbsp; 
  
#  <center>The Umpire Model - Data</center>
## <center>Can we use Major League Baseball Statcast pitching metrics to classify balls and strikes?</center>
##### <center>University of Colorado Boulder | Introduction to Machine Learning: Supervised Learning - Final Project</center>
##### <center>Jeremy Hurley | October, 2024</center>

&nbsp;  

---  

&nbsp;  
  

This notebook will import, clean and save a dataset for use in my final project.  
  
See the main notebook *'The Umpire Model'* for the remainder of the project.

# Data Acquisition:  
The pybaseball Python package was developed by James LeDoux. It pulls data from multiple data sources, including FanGraphs, Baseball Reference and Baseball Savant. It contains a vast amount of data from every MLB game played in the modern era. For more information on pybaseball, find it's GitHub repo [here](https://github.com/jldbc/pybaseball)  

This project starts by pulling all statcast data from the 2023 season.

In [1]:
import pybaseball as pb
import pandas as pd

# Pull statcast data for the entire 2023 MLB season
data = pb.statcast(start_dt='2023-03-30', end_dt='2023-10-01')

# Look at first few rows of data.
data.head()

This is a large query, it may take a moment to complete


C:\Users\hurle\OneDrive\MSDS_Colorado\DTSA 550X - CS Pathway\DTSA 5509 - Intro to Machine Learning\FinalProject\env\Lib\site-packages\pybaseball\statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
  0%|                                                                        | 0/186 [00:00<?, ?it/s]C:\Users\hurle\OneDrive\MSDS_Colorado\DTSA 550X - CS Pathway\DTSA 5509 - Intro to Machine Learning\FinalProject\env\Lib\

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length
2404,CH,2023-10-01,89.0,-2.8,5.59,"Robertson, Nick",677008,687798,field_out,hit_into_play,...,1,1,6,Infield shade,Standard,250,0.0,-0.09,<NA>,<NA>
2483,FF,2023-10-01,96.9,-2.4,5.9,"Robertson, Nick",677008,687798,NaN,foul,...,1,1,6,Infield shade,Standard,211,0.0,0.0,<NA>,<NA>
2627,CH,2023-10-01,90.0,-2.93,5.56,"Robertson, Nick",677008,687798,NaN,ball,...,1,1,6,Infield shade,Standard,250,0.0,0.016,<NA>,<NA>
2644,ST,2023-10-01,82.2,-3.09,5.55,"Robertson, Nick",677008,687798,NaN,ball,...,1,1,6,Infield shade,Standard,58,0.0,0.009,<NA>,<NA>
2791,CH,2023-10-01,89.2,-2.87,5.58,"Robertson, Nick",677008,687798,NaN,swinging_strike,...,1,1,6,Infield shade,Standard,257,0.0,-0.023,<NA>,<NA>


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 717945 entries, 2404 to 4456
Data columns (total 94 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   pitch_type                       717674 non-null  object        
 1   game_date                        717945 non-null  datetime64[ns]
 2   release_speed                    717673 non-null  Float64       
 3   release_pos_x                    717674 non-null  Float64       
 4   release_pos_z                    717674 non-null  Float64       
 5   player_name                      717945 non-null  object        
 6   batter                           717945 non-null  Int64         
 7   pitcher                          717945 non-null  Int64         
 8   events                           183773 non-null  object        
 9   description                      717945 non-null  object        
 10  spin_dir                         0 non-null     

&nbsp;  
  
The previous two cells show that we have 717,945 rows and 94 columns of data.  
  
Next step is to clean and filter the data.

&nbsp;  

---  

&nbsp;  
  

In [4]:
# get all unique values of the 'description' column to see how pybaseball categorizes the outcome of each pitch
print(data['description'].unique())

['hit_into_play' 'foul' 'ball' 'swinging_strike' 'called_strike'
 'foul_tip' 'blocked_ball' 'swinging_strike_blocked' 'hit_by_pitch'
 'foul_bunt' 'missed_bunt' 'pitchout' 'bunt_foul_tip']


# Data Cleaning:  

As you can see above, statcast data on the entire 2023 season results in a massive amount of data. My approach to cleaning the data will be as follows:  

1. Remove columns that are not relevant to our model, including player and score information.
   
*I will keep situational information like the count and baserunners because I think further work can explore whether or not the strike zone changes based on the count (3-0 versus 0-2) or if there are runners on base. This will be discussed in the Discussion and Conclusions section.*  

2. Remove rows where the outcome of the pitch was anything except for a ball or called_strike.

*We can't judge a pitch to be a ball or strike by whether or not the batter swung at it. This project is meant to analyze how an umpire calls balls and strikes, therefore, we'll need to remove instances like 'swinging_strike', 'foul', ect.*  

3. Remove rows with missing values in columns that will be key to our model.

*Will start with 'plate_x', 'plate_z' and 'type', then analyze the state of other columns to see if further action is needed.*  

Finally, considering the size of the initial dataset and the amount of time it took to download, I'll save the cleaned data set to a .csv file for future use.

In [7]:
# Filter data for relevant columns
cleaned_data = data[['plate_x', 'plate_z', 'pitch_type', 'zone', 'sz_top', 'sz_bot', 'type', 'release_speed', 'release_pos_x', 'release_pos_z', 'release_pos_y', 'description', 'stand', 'p_throws', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'pitch_name']]

# Look at cleaned data 
cleaned_data.head()

,plate_x,plate_z,pitch_type,zone,sz_top,sz_bot,type,release_speed,release_pos_x,release_pos_z,...,strikes,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,pitch_name
2404,0.33,2.01,CH,9,3.81,1.74,X,89.0,-2.8,5.59,...,2,-1.53,0.33,0.33,2.01,<NA>,<NA>,<NA>,2,Changeup
2483,0.09,2.71,FF,5,3.81,1.74,S,96.9,-2.4,5.9,...,2,-0.76,1.36,0.09,2.71,<NA>,<NA>,<NA>,2,4-Seam Fastball
2627,-0.24,0.53,CH,13,3.71,1.78,B,90.0,-2.93,5.56,...,2,-1.65,0.36,-0.24,0.53,<NA>,<NA>,<NA>,2,Changeup
2644,0.81,0.49,ST,14,3.78,1.74,B,82.2,-3.09,5.55,...,2,1.43,0.28,0.81,0.49,<NA>,<NA>,<NA>,2,Sweeper
2791,-0.78,2.05,CH,7,3.81,1.74,S,89.2,-2.87,5.58,...,1,-1.49,0.31,-0.78,2.05,<NA>,<NA>,<NA>,2,Changeup


In [8]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 717945 entries, 2404 to 4456
Data columns (total 25 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   plate_x        717674 non-null  Float64
 1   plate_z        717674 non-null  Float64
 2   pitch_type     717674 non-null  object 
 3   zone           717674 non-null  Int64  
 4   sz_top         717674 non-null  Float64
 5   sz_bot         717674 non-null  Float64
 6   type           717945 non-null  object 
 7   release_speed  717673 non-null  Float64
 8   release_pos_x  717674 non-null  Float64
 9   release_pos_z  717674 non-null  Float64
 10  release_pos_y  717674 non-null  Float64
 11  description    717945 non-null  object 
 12  stand          717945 non-null  object 
 13  p_throws       717945 non-null  object 
 14  balls          717945 non-null  Int64  
 15  strikes        717945 non-null  Int64  
 16  pfx_x          717674 non-null  Float64
 17  pfx_z          717674 non-null  F

In [12]:
# Filter out rows where the outcome was not a called ball or called strike
cleaned_data = cleaned_data[cleaned_data['description'].isin(['ball', 'called_strike'])]
#Encode 'type' column as numeric: 1 for strike, 0 for balls
cleaned_data['type'] = cleaned_data['type'].apply(lambda x: 1 if x == 'S' else 0)

# Check the data
cleaned_data.head()

,plate_x,plate_z,pitch_type,zone,sz_top,sz_bot,type,release_speed,release_pos_x,release_pos_z,...,strikes,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,pitch_name
2627,-0.24,0.53,CH,13,3.71,1.78,0,90.0,-2.93,5.56,...,2,-1.65,0.36,-0.24,0.53,<NA>,<NA>,<NA>,2,Changeup
2644,0.81,0.49,ST,14,3.78,1.74,0,82.2,-3.09,5.55,...,2,1.43,0.28,0.81,0.49,<NA>,<NA>,<NA>,2,Sweeper
3228,-1.84,4.08,ST,11,3.07,1.51,0,79.4,-2.96,5.67,...,2,1.66,-0.2,-1.84,4.08,<NA>,<NA>,<NA>,1,Sweeper
3391,1.13,0.97,ST,14,3.1,1.46,0,82.9,-2.82,5.66,...,2,1.16,0.01,1.13,0.97,<NA>,<NA>,<NA>,1,Sweeper
3536,-0.65,2.34,ST,4,3.07,1.48,1,82.0,-3.0,5.69,...,1,1.45,0.06,-0.65,2.34,<NA>,<NA>,<NA>,1,Sweeper


In [15]:
# Remove rows with missing critical info
cleaned_data = cleaned_data.dropna(subset=['plate_x', 'plate_z', 'type'])
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 358481 entries, 2627 to 4456
Data columns (total 25 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   plate_x        358481 non-null  Float64
 1   plate_z        358481 non-null  Float64
 2   pitch_type     358481 non-null  object 
 3   zone           358481 non-null  Int64  
 4   sz_top         358481 non-null  Float64
 5   sz_bot         358481 non-null  Float64
 6   type           358481 non-null  int64  
 7   release_speed  358480 non-null  Float64
 8   release_pos_x  358481 non-null  Float64
 9   release_pos_z  358481 non-null  Float64
 10  release_pos_y  358481 non-null  Float64
 11  description    358481 non-null  object 
 12  stand          358481 non-null  object 
 13  p_throws       358481 non-null  object 
 14  balls          358481 non-null  Int64  
 15  strikes        358481 non-null  Int64  
 16  pfx_x          358481 non-null  Float64
 17  pfx_z          358481 non-null  F


&nbsp;  
  
---  

  
# Data Save:
As you can see in the above cell, we've cleaned the original 94 columns X 717945 rows worth of data to 25 columns X 358481 rows of data. We've filtered out unneeded columns and rows and removed rows with missing data. Further examination of the .info() above shows that, with the exception of on_base columns, which we would expect to be incomplete, all other rows are complete.  

I will now save this dataset as a CSV file to have easier access to it in the future and continue to project in a new notebook.

In [16]:
# Save the dataset to a CSV file
cleaned_data.to_csv('2023_strike_ball_data.csv', index=False)